In [1]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
import re
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras.layers import Conv1D, Flatten, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import matplotlib.pyplot as plt
import re
import string
import numpy as np 
import pandas as pd 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

In [2]:
train_data = pd.read_csv('toxic_train.csv')
test_data= pd.read_csv('toxic_test.csv')

In [5]:
train_data.head()

,Unnamed: 0,comment_text,toxic
0,0,Explanation\r\nWhy the edits made under my use...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\r\nMore\r\nI can't make any real suggestions...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
test_data.head()

,Unnamed: 0,comment_text,toxic
0,5,Thank you for understanding. I think very high...,0
1,7,:Dear god this site is horrible.,0
2,11,"""::: Somebody will invariably try to add Relig...",0
3,13,""" \r\n\r\n It says it right there that it IS a...",0
4,14,""" \r\n\r\n == Before adding a new product to t...",0


In [7]:
train_data.isna().sum()

Unnamed: 0      0
comment_text    0
toxic           0
dtype: int64

In [8]:
test_data.isna().sum()

Unnamed: 0      0
comment_text    0
toxic           0
dtype: int64

In [10]:
def cleanTextAndTokenize(text):
        stop_words= stopwords.words('english')
        lem = WordNetLemmatizer()
        text = re.sub('[^A-Za-z]+', ' ', text).lower()
        text= "".join([i.lower() for i in text if i not in string.punctuation])
        tokens = word_tokenize(text)
        cleanText = ''
        for token in tokens:
            if(token not in stop_words):
                cleanText+=lem.lemmatize(token) + " "
        return cleanText

In [12]:
train_data['clean_text'] = train_data['comment_text'].apply(cleanTextAndTokenize)
test_data['clean_text'] = test_data['comment_text'].apply(cleanTextAndTokenize)

In [13]:
x_train = train_data['clean_text']
x_test = test_data['clean_text']

In [14]:
y_train = train_data['toxic']
y_test = test_data['toxic']

In [15]:
print(x_train.shape, x_test.shape)

(159571,) (63978,)


In [16]:
print(y_train.shape,y_test.shape)

(159571,) (63978,)


# 1D CNN using Keras Tokenizer

In [19]:
vocab_size = 1000
max_len =1000
batch_size = 32
embedding_dims =10
filters = 16
ker_size = 3 # kernel size
hidden_dims = 250
epochs = 10

In [20]:
tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train)

In [21]:
x_train = tokenizer.texts_to_matrix(x_train)
x_test = tokenizer.texts_to_matrix(x_test)

In [22]:
x_train = sequence.pad_sequences(x_train,maxlen=max_len)
x_test = sequence.pad_sequences(x_test,maxlen=max_len)

In [23]:
model = Sequential()
# First we add an embedding layer 
model.add(Embedding(vocab_size,embedding_dims,input_length=max_len)) 
# Adding a 1D convolutional Layer
model.add(Conv1D(filters, ker_size, padding='valid', activation='relu'))
# Max Pooling the Convolutions
model.add(MaxPooling1D())
# Again Computing the Convolutions
model.add(Flatten())
model.add(Dense(hidden_dims, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.fit(x_train,y_train, batch_size=batch_size, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
4987/4987 [==============================] - 129s 26ms/step - loss: 0.1733 - accuracy: 0.9432 - val_loss: 0.1916 - val_accuracy: 0.9293
Epoch 2/5
4987/4987 [==============================] - 130s 26ms/step - loss: 0.1473 - accuracy: 0.9511 - val_loss: 0.1919 - val_accuracy: 0.9295
Epoch 3/5
4987/4987 [==============================] - 131s 26ms/step - loss: 0.1369 - accuracy: 0.9535 - val_loss: 0.1892 - val_accuracy: 0.9311
Epoch 4/5
4987/4987 [==============================] - 128s 26ms/step - loss: 0.1229 - accuracy: 0.9573 - val_loss: 0.2038 - val_accuracy: 0.9293
Epoch 5/5
4987/4987 [==============================] - 125s 25ms/step - loss: 0.1047 - accuracy: 0.9632 - val_loss: 0.2296 - val_accuracy: 0.9257


In [ ]:
# 95% accuracy on train data. 92.5% on test data

In [157]:
vectorizer = TextVectorization(max_tokens=1000, output_sequence_length=400)
vectorizer.adapt(np.array(x_train))
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [158]:
embeddings_index = {}
with open('glove.6B.50d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [159]:
num_tokens = len(voc) + 2
embedding_dim = 50
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 972 words (28 misses)


In [160]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras import initializers
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=initializers.Constant(embedding_matrix),
    trainable=False,
)

In [161]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras import Model
int_sequences_input = Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
# x = layers.Conv1D(128, 5, activation="relu")(x)
# x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(3, activation="softmax")(x)
model = Model(int_sequences_input, preds)
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_5 (Embedding)      (None, None, 50)          50100     
_________________________________________________________________
conv1d_26 (Conv1D)           (None, None, 128)         32128     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_27 (Conv1D)           (None, None, 128)         82048     
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               1651

In [162]:
x_train = vectorizer(np.array([[s] for s in x_train])).numpy()
x_test = vectorizer(np.array([[s] for s in x_test])).numpy()

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
31/41 [=====================>........] - ETA: 1s - loss: 1.0900 - acc: 0.3765